In [112]:
import os
from mistralai import Mistral
from dotenv import load_dotenv
load_dotenv()
import os

In [113]:
prompts = {
    "Johnson": {
        "medical_notes": "A 60-year-old male patient, Mr. Johnson, presented with symptoms of increased thirst, frequent urination, fatigue, and unexplained weight loss. Upon evaluation, he was diagnosed with diabetes, confirmed by elevated blood sugar levels. Mr. Johnson's weight is 210 lbs. He has been prescribed Metformin to be taken twice daily with meals. It was noted during the consultation that the patient is a current smoker. ",
        "golden_answer": {
            "age": 60,
            "gender": "male",
            "diagnosis": "diabetes",
            "weight": 210,
            "smoking": "yes",
        },
    },
    "Smith": {
        "medical_notes": "Mr. Smith, a 55-year-old male patient, presented with severe joint pain and stiffness in his knees and hands, along with swelling and limited range of motion. After a thorough examination and diagnostic tests, he was diagnosed with arthritis. It is important for Mr. Smith to maintain a healthy weight (currently at 150 lbs) and quit smoking, as these factors can exacerbate symptoms of arthritis and contribute to joint damage.",
        "golden_answer": {
            "age": 55,
            "gender": "male",
            "diagnosis": "arthritis",
            "weight": 150,
            "smoking": "yes",
        },
    },
}

In [114]:
def run_mistral(user_message, model="mistral-large-latest"):
    client = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))
    messages = [{"role": "user", "content": user_message}]
    chat_response = client.chat.complete(
        model=model,
        messages=messages,
        response_format={"type": "json_object"},
    )
    return chat_response.choices[0].message.content


# define prompt template
prompt_template = """
Extract information from the following medical notes:
{medical_notes}

Return json format with the following JSON schema: 

{{
        "age": {{
            "type": "integer"
        }},
        "gender": {{
            "type": "string",
            "enum": ["male", "female", "other"]
        }},
        "diagnosis": {{
            "type": "string",
            "enum": ["migraine", "diabetes", "arthritis", "acne", "common cold"]
        }},
        "weight": {{
            "type": "integer"
        }},
        "smoking": {{
            "type": "string",
            "enum": ["yes", "no"]
        }},
        
}}
"""

In [115]:
import json

def compare_json_objects(obj1, obj2):
    total_fields = 0
    identical_fields = 0
    common_keys = set(obj1.keys()) & set(obj2.keys())
    for key in common_keys:
        identical_fields += obj1[key] == obj2[key]
    percentage_identical = (identical_fields / max(len(obj1.keys()), 1)) * 100
    return percentage_identical

In [116]:
accuracy_rates = []

# for each test case
for name in prompts:

    # define user message
    user_message = prompt_template.format(medical_notes=prompts[name]["medical_notes"])

    # run LLM
    response = json.loads(run_mistral(user_message))

    # calculate accuracy rate for this test case
    accuracy_rates.append(
        compare_json_objects(response, prompts[name]["golden_answer"])
    )

# calculate accuracy rate across test cases
sum(accuracy_rates) / len(accuracy_rates)

100.0

## Another Mistral Use-case

In [12]:
#!pip -q install langchain
!pip -q install huggingface_hub
!pip -q install  git+https://github.com/huggingface/transformers # need to install from github
!pip -q install kor
#!pip install pandas --upgrade
!pip install Pathlib
%pip install protobuf==3.20.0

Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip show langchain

Name: langchain
Version: 0.2.15
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/mattsalomon/miniconda3/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


In [14]:
!pip show kor

Name: kor
Version: 2.0.0
Summary: Extract information with LLMs from text
Home-page: https://www.github.com/eyurtsev/kor
Author: Eugene Yurtsev
Author-email: eyurtsev@gmail.com
License: MIT
Location: /Users/mattsalomon/miniconda3/lib/python3.10/site-packages
Requires: langchain-core, pandas, pydantic
Required-by: 


In [15]:
!pip show protobuf

Name: protobuf
Version: 3.20.3
Summary: Protocol Buffers
Home-page: https://developers.google.com/protocol-buffers/
Author: 
Author-email: 
License: BSD-3-Clause
Location: /Users/mattsalomon/miniconda3/lib/python3.10/site-packages
Requires: 
Required-by: googleapis-common-protos, onnxruntime, opentelemetry-proto, tensorboard, tensorflow, tensorflow-datasets, tensorflow-metadata


In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
from dotenv import load_dotenv
import os
import textwrap
import enum
import re
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

from operator import itemgetter
from typing import List, Union, Optional

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import GenerationConfig

import langchain
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate,SystemMessagePromptTemplate,AIMessagePromptTemplate,HumanMessagePromptTemplate
from langchain.prompts import StringPromptTemplate

from langchain import LLMChain
from langchain.llms import HuggingFaceHub, HuggingFacePipeline
from langchain.schema.runnable import ConfigurableField

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from pydantic import BaseModel, Field, validator
from kor import extract_from_documents, from_pydantic

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
load_dotenv()
pd.set_option('display.max_colwidth', 500)
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.environ['hf_token']

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


## Load Dataset
**Restaurant Reviews with following aspects**

- Ambience
- Anecdotes
- Food
- Price
- Staff
- Miscellaneous

In [3]:
review_domain = 'restaurant'

input_dir = Path(f'/Users/mattsalomon/Downloads/datasets/{review_domain}')
data_dir = Path('/Users/mattsalomon/Downloads/datasets/')

In [4]:
def load_dataset(filename):
    f = open(filename, 'r', encoding='utf-8')
    all_reviews = f.readlines()
    print('Total Reviews: ', len(all_reviews))
    f.close()

    sentences = []
    for i,review in enumerate(all_reviews):
        sentences.append(review.strip('\n'))

    col = 'labels' if 'test_label' in str(filename) else 'text_org'

    df = pd.DataFrame({col:sentences})
    return df

In [5]:
df_label = load_dataset(input_dir/'test_label.txt')
print(df_label.shape)

all_labels = list(set(df_label['labels'].str.split(' ').sum()))
print(all_labels)
drop_labels = ['Positive', 'Neutral']
all_labels = sorted([label for label in all_labels if label not in drop_labels])

drop_index = pd.Series(False, index=df_label.index)
for label in drop_labels:
    drop_index = drop_index | df_label['labels'].str.contains(label)
    
print(all_labels)

df_label = df_label[~drop_index].copy()
print(df_label.shape)

for label in all_labels:
    df_label[label] = np.where(df_label['labels'].str.contains(label), 1, 0)
print(df_label.shape)

df_label.head()

Total Reviews:  3328
(3328, 1)
['Miscellaneous', 'Ambience', 'Anecdotes', 'Price', 'Staff', 'Food', 'Positive', 'Neutral']
['Ambience', 'Anecdotes', 'Food', 'Miscellaneous', 'Price', 'Staff']
(3315, 1)
(3315, 7)


,labels,Ambience,Anecdotes,Food,Miscellaneous,Price,Staff
0,Food Ambience,1,0,1,0,0,0
1,Staff,0,0,0,0,0,1
2,Ambience,1,0,0,0,0,0
3,Miscellaneous,0,0,0,1,0,0
4,Miscellaneous,0,0,0,1,0,0


In [6]:
df = load_dataset(input_dir/'test.txt')
print(df.shape)

df = df[~drop_index].copy()
print(df.shape)

df = pd.concat([df, df_label], axis=1)

df.head()

Total Reviews:  3328
(3328, 1)
(3315, 1)


,text_org,labels,Ambience,Anecdotes,Food,Miscellaneous,Price,Staff
0,Always a fun place ... the food is deeelish !,Food Ambience,1,0,1,0,0,0
1,"The staff is n't the friendliest or most competent , and I am stickler for service , but everything else about this place makes up for it .",Staff,0,0,0,0,0,1
2,"Great for groups , great for a date , great for early brunch or a nightcap .",Ambience,1,0,0,0,0,0
3,Another great place to take out-of-towners !,Miscellaneous,0,0,0,1,0,0
4,: ),Miscellaneous,0,0,0,1,0,0


In [7]:
df[all_labels].mean()

Ambience         0.113424
Anecdotes        0.129412
Food             0.370136
Miscellaneous    0.276621
Price            0.089894
Staff            0.166817
dtype: float64

## Setup LLM: Mistral 7B

In [8]:
# !pip install transformers==4.20
# %pip install sentencepiece

In [9]:
# repo_id = "Open-Orca/Mistral-7B-OpenOrca"

# model = AutoModelForCausalLM.from_pretrained(repo_id, torch_dtype="auto")
# tokenizer = AutoTokenizer.from_pretrained(repo_id, torch_dtype="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# pipe = pipeline(
#     "text-generation",
#     model=model, 
#     tokenizer=tokenizer, 
#     max_new_tokens = 50,
#     device=device,
#     pad_token_id=tokenizer.eos_token_id,
#     temperature=0,
# )

# llm = HuggingFacePipeline(pipeline=pipe)

In [11]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x36d7dff40>)

In [14]:
openai_api_key=os.environ['OPENAI_API_KEY']

In [16]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens=2000,
    openai_api_key=openai_api_key
)

## Setup Aspect Extractor

In [17]:
class Aspects(enum.Enum):
    staff = "Staff"
    ambience = "Ambience"
    anecdotes = "Anecdotes"
    food = "Food"
    price = 'Price'
    miscellaneous = 'Miscellaneous'
    
class ReviewABSA(BaseModel):
    aspect: List[Aspects] = Field(
        description=f"The key features that customers are talking about in their {review_domain} reviews.",
        examples=[
            ("The Singapore Mai Fun had NO curry flavor whatsoever.", "Food"),
            ("Go for it !", "Miscellaneous"),
            ("While the staff at this little bistro is very friendly , I have never experienced more incompetency.", "Staff"),
            ("Went on a double date with friend and his girlfriend for a few drinks and appetizers .", "Anecdotes"),
            ("When you enter , you feel like you have entered your mom 's kitchen .", "Ambience"),
            ("Pricey , but worth a try , at least once .", "Price")
        ],
        many=True,
    )

In [18]:
instruction_template = PromptTemplate(
    input_variables=["format_instructions", "type_description"],
    template=(
        "<|im_start|>system\nYour goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.\n\n"
        "{type_description}\n<|im_end|>\n<|im_start|>user\n{format_instructions}\n"
    ),
)

In [19]:
#from langchain.chains import create_extraction_chain
schema, validator = from_pydantic(ReviewABSA)   
chain = create_extraction_chain(
    llm, schema, validator=validator, 
    encoder_or_encoder_class="json",
    instruction_template=instruction_template
)

In [20]:
print(chain.get_prompts()[0].format_prompt(text="[user input]").to_string())

<|im_start|>system
Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

reviewabsa: { // 
 aspect: Array<"Staff" | "Ambience" | "Anecdotes" | "Food" | "Price" | "Miscellaneous"> // The key features that customers are talking about in their restaurant reviews.
}
```

<|im_end|>
<|im_start|>user
Please output the extracted information in JSON format. Do not output anything except for the extracted information. Do not add any clarifying information. Do not add any fields that are not in the schema. If the text contains attributes that do not appear in the schema, please ignore them. All output must be in JSON format and follow the schema specified above. Wrap the JSON in <json> tags.


Input: The Singapore Mai Fun had NO curry flavor whatsoever.
Output: <json>{"re

In [21]:
#print(chain.prompt.format_prompt(text="[user input]").to_string())

## Aspect Extraction

In [29]:
sample = 100
all_labels_sampled=all_labels[0:100]
reviews_sampled=reviews[0:100]
aspects_sampled=aspects[0:100]

In [30]:
%%time

reviews = df['text_org'].values.tolist()

responses = []
aspects = []
    
for text in tqdm(reviews_sampled):
    response = chain.invoke(input=text+'<|im_end|>\n')
    responses.append(response)
    aspects.append(response['data']['reviewabsa']['aspect'] if len(response['data'].keys())>0 
                   else ['Miscellaneous'])

len(reviews), len(responses), len(aspects)

100%|█████████████████████████████████████████| 100/100 [01:08<00:00,  1.45it/s]

CPU times: user 4.43 s, sys: 256 ms, total: 4.68 s
Wall time: 1min 8s


(3315, 100, 100)

In [33]:
df_test_pred = pd.DataFrame(columns=[f'predicted_{label}' for label in all_labels_sampled])
df_test_pred['predicted_labels'] = [' '.join(aspect) for aspect in aspects_sampled]
for label in all_labels_sampled:
    df_test_pred[f'predicted_{label}'] = np.where(df_test_pred['predicted_labels'].str.contains(label), 1, 0)
df_test_pred.index = df.iloc[0:sample].index
print(df_test_pred.shape)

(100, 7)


## Evaluate Results

In [36]:
df_final = pd.concat([df.iloc[0:sample], df_test_pred], axis=1)
print(df_final.shape)
df_final = df_final[['text_org', 'labels', 'predicted_labels']+all_labels_sampled+[f'predicted_{label}' for label in all_labels_sampled]] 
df_final.head()

(100, 15)


,text_org,labels,predicted_labels,Ambience,Anecdotes,Food,Miscellaneous,Price,Staff,predicted_Ambience,predicted_Anecdotes,predicted_Food,predicted_Miscellaneous,predicted_Price,predicted_Staff
0,Always a fun place ... the food is deeelish !,Food Ambience,Ambience Food,1,0,1,0,0,0,1,0,1,0,0,0
1,"The staff is n't the friendliest or most competent , and I am stickler for service , but everything else about this place makes up for it .",Staff,Staff Miscellaneous,0,0,0,0,0,1,0,0,0,1,0,1
2,"Great for groups , great for a date , great for early brunch or a nightcap .",Ambience,Miscellaneous,1,0,0,0,0,0,0,0,0,1,0,0
3,Another great place to take out-of-towners !,Miscellaneous,Miscellaneous,0,0,0,1,0,0,0,0,0,1,0,0
4,: ),Miscellaneous,Miscellaneous,0,0,0,1,0,0,0,0,0,1,0,0


In [37]:
interested_labels = ['Ambience', 'Food', 'Price', 'Staff']
for label in interested_labels:
    print('Label: ', label)
    y_true = df_final[label]
    y_pred = df_final[f'predicted_{label}']
    print('Accuracy: ', np.round(accuracy_score(y_true, y_pred),3))
    print('F1 Score: ', np.round(f1_score(y_true, y_pred),3))
    print('Precision: ', np.round(precision_score(y_true, y_pred),3))
    print('Recall: ', np.round(recall_score(y_true, y_pred),3))
    print('***************')

Label:  Ambience
Accuracy:  0.87
F1 Score:  0.552
Precision:  0.667
Recall:  0.471
***************
Label:  Food
Accuracy:  0.88
F1 Score:  0.714
Precision:  0.833
Recall:  0.625
***************
Label:  Price
Accuracy:  0.99
F1 Score:  0.8
Precision:  1.0
Recall:  0.667
***************
Label:  Staff
Accuracy:  0.97
F1 Score:  0.914
Precision:  1.0
Recall:  0.842
***************


In [38]:
other_labels = ['Anecdotes', 'Miscellaneous']
for label in other_labels:
    print('Label: ', label)
    y_true = df_final[label]
    y_pred = df_final[f'predicted_{label}']
    print('Accuracy: ', np.round(accuracy_score(y_true, y_pred),3))
    print('F1 Score: ', np.round(f1_score(y_true, y_pred),3))
    print('Precision: ', np.round(precision_score(y_true, y_pred),3))
    print('Recall: ', np.round(recall_score(y_true, y_pred),3))
    print('***************')

Label:  Anecdotes
Accuracy:  0.81
F1 Score:  0.0
Precision:  0.0
Recall:  0.0
***************
Label:  Miscellaneous
Accuracy:  0.61
F1 Score:  0.621
Precision:  0.457
Recall:  0.97
***************


## Another Misral AI example:

In [74]:
import pandas as pd

# Define the dataset structure
data = {
    "Text": [],
    "Person": [],
    "Date": [],
    "Location": [],
    "Event": [],
    "Organization": []
}

# Entry 1
data["Text"].append("John Doe attended the annual conference in New York on September 10, 2023, hosted by Microsoft.")
data["Person"].append("John Doe")
data["Date"].append("September 10, 2023")
data["Location"].append("New York")
data["Event"].append("Annual conference")
data["Organization"].append("Microsoft")

# Entry 2
data["Text"].append("Jane Smith delivered a keynote speech at the AI Summit in San Francisco on March 15, 2022, organized by Google.")
data["Person"].append("Jane Smith")
data["Date"].append("March 15, 2022")
data["Location"].append("San Francisco")
data["Event"].append("AI Summit")
data["Organization"].append("Google")

# Entry 3
data["Text"].append("Michael Johnson was invited to the World Economic Forum in Davos on January 21, 2024, as a representative of Tesla.")
data["Person"].append("Michael Johnson")
data["Date"].append("January 21, 2024")
data["Location"].append("Davos")
data["Event"].append("World Economic Forum")
data["Organization"].append("Tesla")

# Entry 4
data["Text"].append("Alice Brown participated in the Global Health Conference in London on July 18, 2021, sponsored by WHO.")
data["Person"].append("Alice Brown")
data["Date"].append("July 18, 2021")
data["Location"].append("London")
data["Event"].append("Global Health Conference")
data["Organization"].append("WHO")

# Entry 5
data["Text"].append("David Wilson joined the Cybersecurity Expo in Berlin on November 2, 2023, held by IBM.")
data["Person"].append("David Wilson")
data["Date"].append("November 2, 2023")
data["Location"].append("Berlin")
data["Event"].append("Cybersecurity Expo")
data["Organization"].append("IBM")

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV (Optional)
df.to_csv('structured_data_dataset.csv', index=False)

# Display the dataset
df

,Text,Person,Date,Location,Event,Organization
0,"John Doe attended the annual conference in New York on September 10, 2023, hosted by Microsoft.",John Doe,"September 10, 2023",New York,Annual conference,Microsoft
1,"Jane Smith delivered a keynote speech at the AI Summit in San Francisco on March 15, 2022, organized by Google.",Jane Smith,"March 15, 2022",San Francisco,AI Summit,Google
2,"Michael Johnson was invited to the World Economic Forum in Davos on January 21, 2024, as a representative of Tesla.",Michael Johnson,"January 21, 2024",Davos,World Economic Forum,Tesla
3,"Alice Brown participated in the Global Health Conference in London on July 18, 2021, sponsored by WHO.",Alice Brown,"July 18, 2021",London,Global Health Conference,WHO
4,"David Wilson joined the Cybersecurity Expo in Berlin on November 2, 2023, held by IBM.",David Wilson,"November 2, 2023",Berlin,Cybersecurity Expo,IBM


In [117]:
# Wrap pattern column in a dictionary
#df["pattern"] = df.apply(lambda x: {"Person": x[0]})
import ast
# Output in JSONL format
jsonl_data =df.to_json(orient='records', lines=True)
jsonl_data
# jsonl_data_dict = ast.literal_eval(jsonl_data)
# # jsonl_data_dict
# {i:[eval(x)]  for i,x in enumerate(jsonl_data.split('\n'))}

'{"Text":"John Doe attended the annual conference in New York on September 10, 2023, hosted by Microsoft.","Person":"John Doe","Date":"September 10, 2023","Location":"New York","Event":"Annual conference","Organization":"Microsoft"}\n{"Text":"Jane Smith delivered a keynote speech at the AI Summit in San Francisco on March 15, 2022, organized by Google.","Person":"Jane Smith","Date":"March 15, 2022","Location":"San Francisco","Event":"AI Summit","Organization":"Google"}\n{"Text":"Michael Johnson was invited to the World Economic Forum in Davos on January 21, 2024, as a representative of Tesla.","Person":"Michael Johnson","Date":"January 21, 2024","Location":"Davos","Event":"World Economic Forum","Organization":"Tesla"}\n{"Text":"Alice Brown participated in the Global Health Conference in London on July 18, 2021, sponsored by WHO.","Person":"Alice Brown","Date":"July 18, 2021","Location":"London","Event":"Global Health Conference","Organization":"WHO"}\n{"Text":"David Wilson joined the C

In [118]:
# define prompt template
prompt_template = """
Extract information from the following medical notes:
{Text}

Return json format with the following JSON schema: 

{{
        "Person": {{
            "type": "string"
        }},
        "Date": {{
            "type": "date",
        }},
        "Location": {{
            "type": "string",
        }},
        "Event": {{
            "type": "string"
        }},
        "Organization": {{
            "type": "string",
        }},
        
}}
"""

In [119]:
# jsonl_data.keys()#[x for x in jsonl_data[i].keys() if x !='Text']

In [120]:
# {    "record1": {
#         "medical_notes":  "the relevant text",
#         "golden_answer": {
#             "Person": 'value',
#             "Date": "value",
#             "Locatoion": "value",
#             "Event": 'value',
#             "Organization": "value",
#         },
#     },
#     "record2": {
#         "text": "the relevant text",
#         "golden_answer": {
#             "Person": 55,
#             "Date": "value",
#             "Locatoion": "value",
#             "Event": 'value',
#             "Organization": "value",
#         },
#     },
# }

In [121]:
import json

# Original string with newline-separated JSON entries
original_string = '''{"Text":"John Doe attended the annual conference in New York on September 10, 2023, hosted by Microsoft.","Person":"John Doe","Date":"September 10, 2023","Location":"New York","Event":"Annual conference","Organization":"Microsoft"}\n{"Text":"Jane Smith delivered a keynote speech at the AI Summit in San Francisco on March 15, 2022, organized by Google.","Person":"Jane Smith","Date":"March 15, 2022","Location":"San Francisco","Event":"AI Summit","Organization":"Google"}\n{"Text":"Michael Johnson was invited to the World Economic Forum in Davos on January 21, 2024, as a representative of Tesla.","Person":"Michael Johnson","Date":"January 21, 2024","Location":"Davos","Event":"World Economic Forum","Organization":"Tesla"}\n{"Text":"Alice Brown participated in the Global Health Conference in London on July 18, 2021, sponsored by WHO.","Person":"Alice Brown","Date":"July 18, 2021","Location":"London","Event":"Global Health Conference","Organization":"WHO"}\n{"Text":"David Wilson joined the Cybersecurity Expo in Berlin on November 2, 2023, held by IBM.","Person":"David Wilson","Date":"November 2, 2023","Location":"Berlin","Event":"Cybersecurity Expo","Organization":"IBM"}\n'''

# Split the string into individual JSON objects
json_entries = original_string.strip().split('\n')

# Parse each JSON string into a dictionary
records = []
for i, entry in enumerate(json_entries):
    parsed_entry = json.loads(entry)
    
    # Create a record for each entry with the desired structure
    record = {
        f"record{i+1}": {
            "Text": parsed_entry["Text"],
            "golden_answer": {
                "Person": parsed_entry["Person"],
                "Date": parsed_entry["Date"],
                "Location": parsed_entry["Location"],
                "Event": parsed_entry["Event"],
                "Organization": parsed_entry["Organization"],
            }
        }
    }
    records.append(record)

# Combine all records into a single dictionary
final_output = {key: value for record in records for key, value in record.items()}

# Convert the final output to JSON string for pretty-printing
final_json = json.dumps(final_output, indent=4)

# Print the final JSON structure
final_json

'{\n    "record1": {\n        "Text": "John Doe attended the annual conference in New York on September 10, 2023, hosted by Microsoft.",\n        "golden_answer": {\n            "Person": "John Doe",\n            "Date": "September 10, 2023",\n            "Location": "New York",\n            "Event": "Annual conference",\n            "Organization": "Microsoft"\n        }\n    },\n    "record2": {\n        "Text": "Jane Smith delivered a keynote speech at the AI Summit in San Francisco on March 15, 2022, organized by Google.",\n        "golden_answer": {\n            "Person": "Jane Smith",\n            "Date": "March 15, 2022",\n            "Location": "San Francisco",\n            "Event": "AI Summit",\n            "Organization": "Google"\n        }\n    },\n    "record3": {\n        "Text": "Michael Johnson was invited to the World Economic Forum in Davos on January 21, 2024, as a representative of Tesla.",\n        "golden_answer": {\n            "Person": "Michael Johnson",\n    

In [ ]:
accuracy_rates = []
def run_mistral(user_message, model="mistral-large-latest"):
    client = Mistral(api_key=os.getenv("MISTRAL_API_KEY"))
    messages = [{"role": "user", "content": user_message}]
    chat_response = client.chat.complete(
        model=model,
        messages=messages,
        response_format={"type": "json_object"},
    )
    return chat_response.choices[0].message.content

def calc_accuracy_rates(data):
    # for each test case
    for i,name in enumerate(data):
        print(data[name])
        # define user message
        user_message = prompt_template.format(Text=data[name])

        # run LLM
        response = json.loads(run_mistral(user_message))

        # calculate accuracy rate for this test case
        accuracy_rates.append(
            compare_json_objects(response, data[name]["golden_answer"])
        )
    return accuracy_rates
# calculate accuracy rate across test cases
accuracy_rates = calc_accuracy_rates(eval(final_json))
sum(accuracy_rates) / len(accuracy_rates)

{'Text': 'John Doe attended the annual conference in New York on September 10, 2023, hosted by Microsoft.', 'golden_answer': {'Person': 'John Doe', 'Date': 'September 10, 2023', 'Location': 'New York', 'Event': 'Annual conference', 'Organization': 'Microsoft'}}
{'Text': 'Jane Smith delivered a keynote speech at the AI Summit in San Francisco on March 15, 2022, organized by Google.', 'golden_answer': {'Person': 'Jane Smith', 'Date': 'March 15, 2022', 'Location': 'San Francisco', 'Event': 'AI Summit', 'Organization': 'Google'}}
{'Text': 'Michael Johnson was invited to the World Economic Forum in Davos on January 21, 2024, as a representative of Tesla.', 'golden_answer': {'Person': 'Michael Johnson', 'Date': 'January 21, 2024', 'Location': 'Davos', 'Event': 'World Economic Forum', 'Organization': 'Tesla'}}


In [105]:
# accuracy_rates = []

# # for each test case
# for name in prompts:

#     # define user message
#     user_message = prompt_template.format(medical_notes=prompts[name]["medical_notes"])

#     # run LLM
#     response = json.loads(run_mistral(user_message))

#     # calculate accuracy rate for this test case
#     accuracy_rates.append(
#         compare_json_objects(response, prompts[name]["golden_answer"])
#     )

# # calculate accuracy rate across test cases
# sum(accuracy_rates) / len(accuracy_rates)